In [1]:
import os
os.getcwd()

'/Users/arun'

In [2]:
os.chdir('/Users/arun/Downloads')

In [3]:
import random


class Sentiment:
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    
    def evenly_distribute(self):
        negative = list(filter(lambda x : x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x : x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

        print(negative[0].text)
        print(len(negative))
        print(len(positive))
        

        
        
        

            
        
        
    

In [4]:
import json
file_name = "./Books_small_10000.json"
reviews = []
with open(file_name) as f:
    for line in f:
        review  = json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
      
reviews[9].sentiment
    
   

'POSITIVE'

### Prep Data


In [42]:
pip install -U scikit-learn


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews, test_size = 0.33,random_state = 42)
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


In [7]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()
test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)



It was just one of those books that never went anywhere. I like books that get your attention in the beginning and not drag out until a quarter way through. I decided to give it an early death - delete!
436
5611
Story is very inaccurate with modern words, phrases and actions.  In the second chapter the author has the bagpipes playing "Amazing Grace" and according to her it is a song as old as time.  As someone who learned to play Amazing Grace on the piano I can state for a fact the song is not old as time. It was not even published until 1779; author has the book set in 1714. 65 years before John Newton wrote and published the songFiona and Juliet speak like they are in the 21 century. Not a young miss in the early 18th century.I have no problem reading about God in books. My problem is when authors take too much leeway and write using modern phrases in historical books.Really, wondering if this author did any 'real' research or just used what she remembered from high school world his

436

#### Bag of words vectorization

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)
print(train_x[0])
print(train_x_vectors[0].toarray())


Donna Tartt is a fine writer.  The book takes place in 3 cities, Manhattan (no outer boroughs in Tartt's New York), Las Vegas, and Amsterdam.  I've been in all 3 cities, and Tartt gets each of them -- Manhattan with its elegant towers and parks and multi-ethnic neighborhoods; Vegas where subdivisions fade into the desert; and Amsterdam with narrow houses, canals, and labyrinthine streets.  But the description goes on for pages and pages (I imagine, since I read it on a Kindle) with nothing happening --just endless word pictures.The book is as full of interesting characters as a Dickens novel, but one never sees inside any of the characters.  Even Theo is a bit of a cipher.  He's constantly taking drugs -- halfway through the book we learn that the pills and heroin keep his paranoia at bay.  Well,, it makes sense that a man caught in a terrorist attack would be paranoid, but how about showing and not telling?  All that verbiage, but not a word until then about Theo's fears and how the d

### Classification

### Linear SVM 

In [9]:
from sklearn import svm
clf_svm = svm.SVC(kernel = "linear")
clf_svm.fit(train_x_vectors, train_y)
clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Decision Tree

In [10]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Evaluation

In [11]:
clf_svm.score(test_x_vectors, test_y)
clf_log.score(test_x_vectors, test_y)

0.8173076923076923

In [12]:
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
#f1_score(test_y, clf_log.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([0.8028169 , 0.        , 0.79310345])

In [13]:
test_y.count(Sentiment.POSITIVE)

208

In [14]:
test_set = ['good',' bad book ','horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

#### Tuning our model (with Grid search)

In [15]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel': ('linear','rbf'), 'C' :(1,4,8,16,32)}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv= 5)
clf.fit(train_x_vectors,train_y)



GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [17]:
print(clf.score(test_x_vectors,test_y))


0.7644230769230769


In [19]:
import pickle
with open('/Users/arun/Downloads/sentiment_classifier.pkl','wb') as f:
    pickle.dump(clf,f)
    

#### Load model

In [20]:
with open('/Users/arun/Downloads/sentiment_classifier.pkl','rb') as f:
    loaded_clf = pickle.load(f)

In [21]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

This is a gorgeous book of fashion illustrations from the top fashion magazines.  I got it because I love the drawings of David Downton and thought they were all his, but he is the editor and has included many other artists besides himself.  It is just beautiful.


array(['NEGATIVE'], dtype='<U8')